In [1]:
from transformer_algebra import *
import torch

model, tokenizer = load_pythia_model("EleutherAI/pythia-14m")


The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [2]:
# 2. Run on next token
T = PromptedTransformer(model, tokenizer, "Once upon")
# 2. Run on next token
x = T(" a")
x

T(embed(' a'))

In [3]:

# 3. Check predictions
predict(x).summary()


"Top 5 predictions:\n  1. ' few' (2.70%, logit=1583.20)\n  2. ' new' (1.01%, logit=1582.22)\n  3. ' time' (0.97%, logit=1582.18)\n  4. ' little' (0.96%, logit=1582.16)\n  5. ' long' (0.88%, logit=1582.08)"

In [4]:
logits(x)[" time"]

<unembed(' time'), T(embed(' a'))> = 1582.18

In [6]:
expand(x)

LN^T((I + ΔB^1 + ΔB^2 T^1 + ΔB^3 T^2 + ΔB^4 T^3 + ΔB^5 T^4 + ΔB^6 T^5)embed(' a'))

In [8]:
contribution(expand(x), logits(x)[" time"])


Contributions to ⟨ time, LN(...)⟩:
  embed(' a'): -1.27 (-0.1%)
  ΔB^1: -2.52 (-0.3%)
  ΔB^2 T^1: +2.95 (+0.3%)
  ΔB^3 T^2: +19.64 (+2.1%)
  ΔB^4 T^3: +5.98 (+0.7%)
  ΔB^5 T^4: +40.43 (+4.4%)
  ΔB^6 T^5: +852.94 (+92.9%)
  Subtotal: 918.15
  Beta term: 664.03
  Total: 1582.18

In [ ]:

# 5. Drill down into significant blocks
for term, value, pct in contrib.top_k(3):
    if hasattr(term, 'expand'):
        print(f"\n{term}:")
        sub = term.expand()
        print(f"  Attention + MLP: {sub}")

In [ ]:
level1 = expand(x)
expand(level1)

In [ ]:
level2 = expand(level1)
expand(level2)

In [ ]:
T = PromptedTransformer(model, tokenizer, "The capital of Ireland")


x = T(" is")

# 3. Check predictions
print(predict(x).summary())

# 4. Expand and analyze
ex = expand(x)
logit = logits(x)[" Dublin"]
contrib = contribution(ex.inner, logit)
print(contrib.summary())

# 5. Drill down into significant blocks
for term, value, pct in contrib.top_k(3):
    if hasattr(term, 'expand'):
        print(f"\n{term}:")
        sub = term.expand()
        print(f"  Attention + MLP: {sub}")